In [1]:
import os
import librosa
# import pydub
from pydub import AudioSegment
import numpy as np
import copy
import random
from scipy import signal as scipy_signal
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

import datetime

In [2]:
dataset_dir = '/home/torooc/dataHDD2/speech_commands_v0.01/'

In [3]:
testing_list_path = dataset_dir + 'testing_list.txt'
validation_list_path = dataset_dir + 'validation_list.txt'

In [4]:
with open(testing_list_path) as f:
    testing_list = f.readlines()
    testing_list = [file.strip() for file in testing_list]
    
with open(validation_list_path) as f:
    validation_list = f.readlines()
    validation_list = [file.strip() for file in validation_list]

In [5]:
command_dirs = os.listdir(dataset_dir)

command_dirs = [direc for direc in command_dirs if os.path.isdir(os.path.join(dataset_dir, direc))]

command_dirs.sort()

command_dirs.remove('_background_noise_')

In [6]:
print(command_dirs)

['bed', 'bird', 'cat', 'dog', 'down', 'eight', 'five', 'four', 'go', 'happy', 'house', 'left', 'marvin', 'nine', 'no', 'off', 'on', 'one', 'right', 'seven', 'sheila', 'six', 'stop', 'three', 'tree', 'two', 'up', 'wow', 'yes', 'zero']


In [7]:
key2label_dict = dict()
label2key_list = list()

key2label_dict[' '] = 0
label2key_list.append(' ')

for i, command in enumerate(command_dirs):
    key2label_dict[command] = i + 1
    label2key_list.append(command)

In [8]:
key2label_dict

{' ': 0,
 'bed': 1,
 'bird': 2,
 'cat': 3,
 'dog': 4,
 'down': 5,
 'eight': 6,
 'five': 7,
 'four': 8,
 'go': 9,
 'happy': 10,
 'house': 11,
 'left': 12,
 'marvin': 13,
 'nine': 14,
 'no': 15,
 'off': 16,
 'on': 17,
 'one': 18,
 'right': 19,
 'seven': 20,
 'sheila': 21,
 'six': 22,
 'stop': 23,
 'three': 24,
 'tree': 25,
 'two': 26,
 'up': 27,
 'wow': 28,
 'yes': 29,
 'zero': 30}

In [9]:
total_wav_file_list = list()

for command_dir in command_dirs:
    wav_list = os.listdir(os.path.join(dataset_dir, command_dir))
    
    wav_list_with_prefix = [os.path.join(command_dir, file_name) for file_name in wav_list]
    
    total_wav_file_list += wav_list_with_prefix

In [10]:
training_list = list(set(total_wav_file_list) - set(testing_list) - set(validation_list))
training_list.sort()


In [11]:
'''
validation_list
testing_list
training_list
'''

'\nvalidation_list\ntesting_list\ntraining_list\n'

In [12]:
audio_metadata = list()

for wav_file_name in total_wav_file_list:
    audio = AudioSegment.from_wav(dataset_dir + wav_file_name)
    fs = audio.frame_rate
    length_in_samples = audio.frame_count()
    
#   normalized_audio_array = np.asarray(audio.get_array_of_samples()) / 2 ** 15
    
#   print("Max: {}, Min: {}".format(max(normalized_audio_array), min(normalized_audio_array)))
    
    audio_metadata.append([fs, length_in_samples])

In [13]:
SR = 16000

class DatasetLoader():
    
    def __init__(self, batch_size, dataset_dir, wav_file_name_list, key2label, label2key):
        self.batch_size = batch_size
        self.dataset_dir = dataset_dir
        self.wav_file_name_list = copy.deepcopy(wav_file_name_list)
        self.sr = SR
        self.nsc_in_ms = 40
#         self.nov_in_ms = self.nsc_in_ms / 2
        self.nov_in_ms = 0
        self.nsc_in_sample = int(self.nsc_in_ms / 1000 * self.sr)
        self.nov_in_sample = int(self.nov_in_ms / 1000 * self.sr)
        
        self.key2label = key2label
        self.label2key = label2key
        
    def shuffle_dataset_order(self):
        random.shuffle(self.wav_file_name_list)
        
    def batch_generator(self):
        
        self.shuffle_dataset_order()
        
        spectrogram_list = list()
        label_list = list()
        
        for i, wav_file_name in enumerate(self.wav_file_name_list):
            
            audio = AudioSegment.from_wav(self.dataset_dir + wav_file_name)
            
            normalized_audio_array = np.asarray(audio.get_array_of_samples()) / 2 ** 15
    
            f, t, Zxx = scipy_signal.stft(normalized_audio_array, fs=self.sr, 
                                          nperseg=self.nsc_in_sample,
                                          noverlap=self.nov_in_sample)
        
            Sxx = np.abs(Zxx)
            
            normalized_spectrogram = (20 * np.log10(np.maximum(Sxx, 1e-8)) + 160) / 160
            spectrogram_list.append(normalized_spectrogram)
            
            keyword = wav_file_name.split('/')[0]
            label = self.key2label[keyword]
            label_list.append(label)
            

            if (i + 1) % self.batch_size == 0 or i + 1 == len(self.wav_file_name_list):
                
                spectrogram_time_step_list = [specgram.shape[1] for specgram in spectrogram_list]
                
                max_time_step = max(spectrogram_time_step_list)
                
                freq_size = spectrogram_list[0].shape[0]
                
                batch = np.zeros([len(spectrogram_list), freq_size, max_time_step])
                
                for j, specgram in enumerate(spectrogram_list):
                    batch[j, :specgram.shape[0], :specgram.shape[1]] = specgram
                
                spectrogram_list = list()
                
#                 print('{}:{}'.format(i, batch.shape[0]))
            
                yield batch, np.asarray(label_list)
                
                label_list = list()
                
#                 break
    
#             break

In [14]:
len(training_list)

51088

In [15]:
dataset_loader_training = DatasetLoader(256, dataset_dir, training_list, key2label_dict, label2key_list)
dataset_loader_testing = DatasetLoader(256, dataset_dir, testing_list, key2label_dict, label2key_list)
dataset_loader_validation = DatasetLoader(256, dataset_dir, validation_list, key2label_dict, label2key_list)

# dataset_loader_testing = DatasetLoader(64, dataset_dir, testing_list, key2label_dict, label2key_list)
# dataset_loader_validation = DatasetLoader(64, dataset_dir, validation_list, key2label_dict, label2key_list)

# dataset_loader_training = DatasetLoader(768, dataset_dir, training_list, key2label_dict, label2key_list)
# dataset_loader_testing = DatasetLoader(768, dataset_dir, testing_list, key2label_dict, label2key_list)
# dataset_loader_validation = DatasetLoader(768, dataset_dir, validation_list, key2label_dict, label2key_list)

In [16]:
# batch_generator = dataset_loader_training.batch_generator()

In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [18]:
# class basic_model(nn.Module):
#     def __init__(self, D_in, H, num_layers, num_labels):
#         super(basic_model, self).__init__()
#         self.fc = torch.nn.Linear(D_in, H)
#         self.relu = torch.nn.ReLU()
#         self.dropout = nn.Dropout(p=0.2)

#         self.gru_layers = nn.ModuleList([nn.GRU(H, int(H / 2), bidirectional=True, batch_first=True) for i in range(num_layers)])

#         self.fc_pred = nn.Linear(H, num_labels)
#         self.log_softmax = nn.LogSoftmax(dim=2)
        
#     def forward(self, input_tensor):
#         # (B, T, F)
#         output_tensor = self.fc(input_tensor)
#         output_tensor = self.relu(output_tensor)
#         output_tensor = self.dropout(output_tensor)
#         # (B, T, H)
#         for layer in self.gru_layers:
#             output_tensor, _ = layer(output_tensor)
            
#         output_tensor = self.fc_pred(output_tensor)

#         output_tensor = self.log_softmax(output_tensor)
        
#         return output_tensor

class basic_conv_model(nn.Module):
#     def __init__(self, first_kernel_size, second_kernel_size, dropout_rate, num_labels):
    def __init__(self):
        super(basic_conv_model, self).__init__()
        self.conv2d_1 = nn.Conv2d(1, 16, (13, 2))
        
        self.conv2d_2 = nn.Conv2d(16, 32, (13, 2))
        
        self.flatten = nn.Flatten()
        
        self.fc = nn.Linear(4800, 31)
        
    def forward(self, input_tensor):
        input_tensor.unsqueeze_(1)
        tensor = self.conv2d_1(input_tensor)
#         print('[After 1st Conv2d]: {}'.format(tensor.shape))
        tensor = F.relu(tensor)
        tensor = F.dropout(tensor, 0.1, training=True)
        tensor = F.max_pool2d(tensor, (3, 2))
#         print('[After 1st maxpool2d]: {}'.format(tensor.shape))
        tensor = self.conv2d_2(tensor)
#         print('[After 2nd Conv2d]: {}'.format(tensor.shape))
        tensor = F.relu(tensor)
        tensor = F.dropout(tensor, 0.1, training=True)
        tensor = F.max_pool2d(tensor, (3, 2))
#         print('[After 2nd maxpool2d]: {}'.format(tensor.shape))
        tensor = self.flatten(tensor)
#         print('[After Flatten]: {}'.format(tensor.shape))
        tensor = self.fc(tensor)
#         print('[After fc]: {}'.format(tensor.shape))
        pred_tensor = F.log_softmax(tensor, dim=-1)
        
        return pred_tensor
    
class basic_rnn_model(nn.Module):
#     def __init__(self, first_kernel_size, second_kernel_size, dropout_rate, num_labels):
    def __init__(self):
        super(basic_rnn_model, self).__init__()
        self.gru_1 = nn.GRU(321, 128, 1)
        self.gru_2 = nn.GRU(128, 64, 1)
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(64 * 26, 31)
        
    def forward(self, input_tensor):
#       input_tensor.unsqueeze_(1)
        # (B, F, T)
        input_tensor.transpose_(0, 2) # (T, F, B)
        input_tensor.transpose_(1, 2) # (T, B, F)
        tensor, _ = self.gru_1(input_tensor)
#         print('[After 1st Conv2d]: {}'.format(tensor.shape))
        tensor = F.relu(tensor)
        tensor = F.dropout(tensor, 0.1, training=True)
#         tensor = F.max_pool2d(tensor, (3, 2))
#         print('[After 1st maxpool2d]: {}'.format(tensor.shape))
        tensor, _ = self.gru_2(tensor)
#         print('[After 2nd Conv2d]: {}'.format(tensor.shape))
        tensor = F.relu(tensor)
        tensor = F.dropout(tensor, 0.1, training=True)
#         tensor = F.max_pool2d(tensor, (3, 2))
#         print('[After 2nd maxpool2d]: {}'.format(tensor.shape))
        tensor.transpose_(0, 1) # (B, T, F)
        tensor = self.flatten(tensor)
#         print('[After Flatten]: {}'.format(tensor.shape))
        tensor = self.fc(tensor)
#         print('[After fc]: {}'.format(tensor.shape))
        pred_tensor = F.log_softmax(tensor, dim=-1)
        
        return pred_tensor
        
    
# model = basic_model(321, 512, 3, 31).float().to(device)
# optimizer = optim.Adam(model.parameters(), lr=1e-4)
# dictation_loss = nn.CTCLoss().to(device)

model = basic_rnn_model().float().to(device)  
optimizer = optim.Adam(model.parameters(), lr=1e-4)
        

In [19]:
# batch_generator = dataset_loader_training.batch_generator()
# next(batch_generator)[0].shape

In [20]:
MAX_EPOCH = 300

for epoch in range(MAX_EPOCH):
    training_loss_list = list()
    batch_generator = dataset_loader_training.batch_generator()
    model.train()
    
    correct_num = 0
    total_num = 0

    for (batch, label) in batch_generator:
        optimizer.zero_grad()
        pred = model(torch.cuda.FloatTensor(batch))
        loss = F.nll_loss(pred, torch.cuda.LongTensor(label))
        loss.backward()
        training_loss_list.append(loss.item())
        optimizer.step()
        
        _, pred_label = torch.max(pred, 1)
        correct_num += int((pred_label == torch.cuda.LongTensor(label)).sum().cpu().numpy())
        total_num += len(label)
        
    acc_train = correct_num / total_num

    testing_loss_list = list()

    batch_generator = dataset_loader_testing.batch_generator()

    model.eval()
    
    correct_num = 0
    total_num = 0

    for (batch, label) in batch_generator:
    #     optimizer.zero_grad()
        pred = model(torch.cuda.FloatTensor(batch))
        loss = F.nll_loss(pred, torch.cuda.LongTensor(label))
#         loss.backward()
        testing_loss_list.append(loss.item())
    #     optimizer.step()
    
        _, pred_label = torch.max(pred, 1)
        correct_num += int((pred_label == torch.cuda.LongTensor(label)).sum().cpu().numpy())
        total_num += len(label)
        
    acc_test = correct_num / total_num

    validation_loss_list = list()

    batch_generator = dataset_loader_validation.batch_generator()
    
    correct_num = 0
    total_num = 0

    for (batch, label) in batch_generator:
    #     optimizer.zero_grad()
        pred = model(torch.cuda.FloatTensor(batch))
        loss = F.nll_loss(pred, torch.cuda.LongTensor(label))
#         loss.backward()
        validation_loss_list.append(loss.item())
    #     optimizer.step()
    
        _, pred_label = torch.max(pred, 1)
        correct_num += int((pred_label == torch.cuda.LongTensor(label)).sum().cpu().numpy())
        total_num += len(label)
        
    acc_valid = correct_num / total_num
    
    training_loss_mean = np.mean(training_loss_list)
    testing_loss_mean = np.mean(testing_loss_list)
    validation_loss_mean = np.mean(validation_loss_list)
    
    print(datetime.datetime.now().time())
    print('#{:03d} [Train: {:0.3f}] [Test: {:0.3f}] [Valid: {:0.3f}]'.format(epoch, training_loss_mean, testing_loss_mean, validation_loss_mean))
    print('#{:03d} [Train: {:0.3f}] [Test: {:0.3f}] [Valid: {:0.3f}]'.format(epoch, acc_train, acc_test, acc_valid))
    

07:31:04.663195
#000 [Train: 3.315] [Test: 3.104] [Valid: 3.068]
#000 [Train: 0.072] [Test: 0.113] [Valid: 0.121]
07:31:48.909657
#001 [Train: 3.005] [Test: 2.939] [Valid: 2.895]
#001 [Train: 0.125] [Test: 0.143] [Valid: 0.145]
07:32:31.330024
#002 [Train: 2.826] [Test: 2.748] [Valid: 2.698]
#002 [Train: 0.164] [Test: 0.182] [Valid: 0.197]
07:33:13.795442
#003 [Train: 2.608] [Test: 2.568] [Valid: 2.499]
#003 [Train: 0.226] [Test: 0.237] [Valid: 0.257]
07:33:57.631346
#004 [Train: 2.434] [Test: 2.426] [Valid: 2.342]
#004 [Train: 0.279] [Test: 0.286] [Valid: 0.304]
07:34:42.012793
#005 [Train: 2.308] [Test: 2.317] [Valid: 2.237]
#005 [Train: 0.318] [Test: 0.320] [Valid: 0.344]
07:35:25.678522
#006 [Train: 2.187] [Test: 2.195] [Valid: 2.116]
#006 [Train: 0.351] [Test: 0.354] [Valid: 0.376]
07:36:10.079446
#007 [Train: 2.077] [Test: 2.091] [Valid: 2.014]
#007 [Train: 0.384] [Test: 0.380] [Valid: 0.403]
07:36:53.808806
#008 [Train: 1.967] [Test: 1.987] [Valid: 1.905]
#008 [Train: 0.415] [Te

08:23:03.532454
#072 [Train: 0.408] [Test: 0.519] [Valid: 0.520]
#072 [Train: 0.878] [Test: 0.848] [Valid: 0.848]
08:23:47.741182
#073 [Train: 0.403] [Test: 0.522] [Valid: 0.509]
#073 [Train: 0.880] [Test: 0.846] [Valid: 0.847]
08:24:30.656679
#074 [Train: 0.398] [Test: 0.517] [Valid: 0.505]
#074 [Train: 0.883] [Test: 0.847] [Valid: 0.850]
08:25:13.192462
#075 [Train: 0.393] [Test: 0.511] [Valid: 0.504]
#075 [Train: 0.883] [Test: 0.850] [Valid: 0.851]
08:25:56.559304
#076 [Train: 0.391] [Test: 0.515] [Valid: 0.495]
#076 [Train: 0.884] [Test: 0.850] [Valid: 0.856]
08:26:39.531301
#077 [Train: 0.383] [Test: 0.496] [Valid: 0.496]
#077 [Train: 0.886] [Test: 0.855] [Valid: 0.854]
08:27:22.390218
#078 [Train: 0.383] [Test: 0.508] [Valid: 0.507]
#078 [Train: 0.887] [Test: 0.852] [Valid: 0.850]
08:28:05.635566
#079 [Train: 0.374] [Test: 0.509] [Valid: 0.499]
#079 [Train: 0.888] [Test: 0.851] [Valid: 0.854]
08:28:48.610404
#080 [Train: 0.376] [Test: 0.496] [Valid: 0.479]
#080 [Train: 0.888] [Te

09:14:51.625374
#144 [Train: 0.211] [Test: 0.423] [Valid: 0.412]
#144 [Train: 0.937] [Test: 0.883] [Valid: 0.884]
09:15:35.124193
#145 [Train: 0.212] [Test: 0.414] [Valid: 0.413]
#145 [Train: 0.936] [Test: 0.890] [Valid: 0.886]
09:16:18.050289
#146 [Train: 0.206] [Test: 0.405] [Valid: 0.396]
#146 [Train: 0.938] [Test: 0.886] [Valid: 0.888]
09:17:01.148316
#147 [Train: 0.204] [Test: 0.397] [Valid: 0.406]
#147 [Train: 0.938] [Test: 0.889] [Valid: 0.888]
09:17:43.766365
#148 [Train: 0.201] [Test: 0.396] [Valid: 0.406]
#148 [Train: 0.939] [Test: 0.889] [Valid: 0.888]
09:18:26.966156
#149 [Train: 0.203] [Test: 0.391] [Valid: 0.404]
#149 [Train: 0.940] [Test: 0.886] [Valid: 0.886]
09:19:10.195067
#150 [Train: 0.200] [Test: 0.396] [Valid: 0.404]
#150 [Train: 0.940] [Test: 0.887] [Valid: 0.885]
09:19:52.817813
#151 [Train: 0.198] [Test: 0.402] [Valid: 0.400]
#151 [Train: 0.941] [Test: 0.889] [Valid: 0.891]
09:20:35.661464
#152 [Train: 0.198] [Test: 0.412] [Valid: 0.398]
#152 [Train: 0.940] [Te

10:06:37.427489
#216 [Train: 0.128] [Test: 0.406] [Valid: 0.404]
#216 [Train: 0.962] [Test: 0.898] [Valid: 0.897]
10:07:19.803398
#217 [Train: 0.126] [Test: 0.380] [Valid: 0.402]
#217 [Train: 0.963] [Test: 0.900] [Valid: 0.896]
10:08:02.121061
#218 [Train: 0.122] [Test: 0.412] [Valid: 0.415]
#218 [Train: 0.965] [Test: 0.896] [Valid: 0.894]
10:08:45.044806
#219 [Train: 0.125] [Test: 0.400] [Valid: 0.406]
#219 [Train: 0.963] [Test: 0.897] [Valid: 0.896]
10:09:27.935463
#220 [Train: 0.124] [Test: 0.404] [Valid: 0.406]
#220 [Train: 0.963] [Test: 0.896] [Valid: 0.898]
10:10:11.393058
#221 [Train: 0.122] [Test: 0.397] [Valid: 0.411]
#221 [Train: 0.963] [Test: 0.899] [Valid: 0.895]
10:10:55.317935
#222 [Train: 0.126] [Test: 0.389] [Valid: 0.405]
#222 [Train: 0.961] [Test: 0.901] [Valid: 0.898]
10:11:38.364744
#223 [Train: 0.119] [Test: 0.416] [Valid: 0.420]
#223 [Train: 0.965] [Test: 0.896] [Valid: 0.896]
10:12:21.233574
#224 [Train: 0.122] [Test: 0.401] [Valid: 0.400]
#224 [Train: 0.964] [Te

10:58:29.595626
#288 [Train: 0.078] [Test: 0.458] [Valid: 0.449]
#288 [Train: 0.978] [Test: 0.895] [Valid: 0.897]
10:59:12.364169
#289 [Train: 0.082] [Test: 0.435] [Valid: 0.437]
#289 [Train: 0.976] [Test: 0.898] [Valid: 0.896]
10:59:55.301177
#290 [Train: 0.081] [Test: 0.486] [Valid: 0.442]
#290 [Train: 0.976] [Test: 0.889] [Valid: 0.896]
11:00:39.497091
#291 [Train: 0.079] [Test: 0.445] [Valid: 0.447]
#291 [Train: 0.977] [Test: 0.896] [Valid: 0.897]
11:01:22.401486
#292 [Train: 0.080] [Test: 0.447] [Valid: 0.467]
#292 [Train: 0.977] [Test: 0.897] [Valid: 0.896]
11:02:05.569457
#293 [Train: 0.081] [Test: 0.423] [Valid: 0.439]
#293 [Train: 0.977] [Test: 0.899] [Valid: 0.893]
11:02:48.274475
#294 [Train: 0.079] [Test: 0.431] [Valid: 0.440]
#294 [Train: 0.978] [Test: 0.902] [Valid: 0.898]
11:03:31.219243
#295 [Train: 0.079] [Test: 0.463] [Valid: 0.438]
#295 [Train: 0.977] [Test: 0.899] [Valid: 0.900]
11:04:13.971549
#296 [Train: 0.076] [Test: 0.455] [Valid: 0.439]
#296 [Train: 0.978] [Te

In [21]:
# label.shape

In [22]:
# output = model(torch.cuda.FloatTensor(batch))

In [23]:
# output.shape

In [24]:
# torch.tensor(batch).unsqueeze_(1).shape

In [25]:
# input_tensor = torch.tensor(batch).to(device).float()
# input_tensor = input_tensor.transpose(1, 2)
# model(input_tensor).shape

In [26]:
# for batch, label_list in batch_generator:
#     print(batch.shape[0])
#     plt.figure(figsize=(5, 1))
#     plt.plot(label_list)
#     plt.show()
    
#     plt.figure(figsize=(6, 18))
# #     plt.imshow(batch[0], aspect='auto')
#     plt.imshow(batch[0])
#     plt.show()
    
#     break